In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [14]:
import numpy as np;
import pandas as pd;
import tensorflow as tf

print(tf.__version__)

1.15.2


In [15]:
# Loading data
data=pd.read_csv('/content/diabetes.csv');

In [16]:
data.shape

(758, 9)

In [7]:
# Naming the columns name
data.columns=['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8','Output']

In [8]:
# Checking the Null values in the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       758 non-null    float64
 1   1       758 non-null    float64
 2   2       758 non-null    float64
 3   3       758 non-null    float64
 4   4       758 non-null    float64
 5   5       758 non-null    float64
 6   6       758 non-null    float64
 7   7       758 non-null    float64
 8   Output  758 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 53.4 KB


In [9]:
#Checking the Data distribution
data.describe()

,0,1,2,3,4,5,6,7,Output
count,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000
mean,-0.407807,0.218208,0.176500,-0.289731,-0.323961,-0.032289,-0.663427,-0.516799,0.654354
std,0.386493,0.306465,0.201420,0.258651,0.375608,0.205508,0.283202,0.400674,0.475893
min,-0.882353,-0.557789,-0.606557,-0.858586,-0.966903,-0.457526,-0.994876,-0.966667,0.000000
25%,-0.764706,-0.005025,0.016393,-0.494949,-0.716312,-0.179583,-0.858241,-0.866667,0.000000
50%,-0.529412,0.165829,0.180328,-0.292929,0.000000,-0.034277,-0.748506,-0.633333,1.000000
75%,0.000000,0.407035,0.311475,0.000000,0.000000,0.087928,-0.532451,-0.233333,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


It seems that data is already normalized as each feature range is between (-1,1)

In [20]:
# Function that creates a blank session
# Used for clearing variables between sessions
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

In [18]:
# Dividing inputs and outputs 
inputs=data.iloc[:,:-1].values
outputs=data.iloc[:,-1].values


In [21]:
# Adding the new axis to output
outputs = outputs[:,np.newaxis]
print(outputs.shape)

(758, 1)


In [22]:
# Splitting the data into training and testing data with 80:20 ratio
from sklearn import  model_selection
data_train, data_test, label_train, label_test = model_selection.train_test_split(inputs, outputs, \
                                                                                 test_size=0.2, random_state = 258)

In [59]:
# Model with Hypothesis one
sess = tf_reset()

def create_model():
    # create inputs
    input_ph = tf.placeholder(dtype=tf.float32, shape=[None, 8]) # Since we know the shape of the data we can manually input
    output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 1])

    # create variables
    W0 = tf.get_variable(name='W0', shape=[8, 20], initializer=tf.contrib.layers.xavier_initializer())
    W1 = tf.get_variable(name='W1', shape=[20, 20], initializer=tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable(name='W2', shape=[20, 1], initializer=tf.contrib.layers.xavier_initializer())

    b0 = tf.get_variable(name='b0', shape=[20], initializer=tf.constant_initializer(0.))
    b1 = tf.get_variable(name='b1', shape=[20], initializer=tf.constant_initializer(0.))
    b2 = tf.get_variable(name='b2', shape=[1], initializer=tf.constant_initializer(0.))

    weights = [W0, W1, W2]
    biases = [b0, b1, b2]
    activations = [tf.nn.relu, tf.nn.relu, None]

    # create computation graph
    layer = input_ph
    for W, b, activation in zip(weights, biases, activations):
        layer = tf.matmul(layer, W) + b
        if activation is not None:
            layer = activation(layer)
    output_pred = layer
    
    return input_ph, output_ph, output_pred
    
input_ph, output_ph, output_pred = create_model()
    
# create loss
mse = tf.reduce_mean(0.5 * tf.square(output_pred - output_ph))

# create optimizer

opt = tf.train.AdamOptimizer().minimize(mse)

# initialize variables
sess.run(tf.global_variables_initializer())
# create saver to save model variables
saver = tf.train.Saver()

# run training
batch_size = 32
for training_step in range(10000):
    # get a random subset of the training data
    indices = np.random.randint(low=0, high=len(data_train), size=batch_size)
    input_batch = data_train[indices]
    output_batch = label_train[indices]
    
    # run the optimizer and get the mse
    _, mse_run = sess.run([opt, mse], feed_dict={input_ph: input_batch, output_ph: output_batch})
    
    # print the mse every so often
    if training_step % 1000 == 0:
        print('{0:04d} mse: {1:.3f}'.format(training_step, mse_run))
        saver.save(sess, '/tmp/model_diabetes.ckpt')

0000 mse: 0.277
1000 mse: 0.081
2000 mse: 0.074
3000 mse: 0.074
4000 mse: 0.041
5000 mse: 0.064
6000 mse: 0.041
7000 mse: 0.062
8000 mse: 0.042
9000 mse: 0.022


In the above model, we have used optimizer=AdamOptimizer and batch size=32

In [61]:
# Prediction of  Result with the Hypothesis 1

sess = tf_reset()

# create the model
input_ph, output_ph, output_pred = create_model()

# restore the saved model
saver = tf.train.Saver()
saver.restore(sess, "/tmp/model_diabetes.ckpt")
predictions= sess.run(output_pred, feed_dict={input_ph: data_test})
pTClass = np.round(predictions)
Predicted_Result = np.ravel(abs(pTClass)).astype(int)
Actual_Result= np.ravel(label_test)
print('Predicted test classes :')
# print(np.ravel(abs(pTClass)))
print(Predicted_Result)
print('Actual test classes:')
print(Actual_Result)


INFO:tensorflow:Restoring parameters from /tmp/model_diabetes.ckpt
Predicted test classes :
[0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0
 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1
 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0
 1 2 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 3 0 1 1 0 0 1
 1 1 1 0]
Actual test classes:
[0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0
 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1
 1 1 1 0]


In [62]:
# Predicting the accuracy
from sklearn import metrics
print(metrics.accuracy_score(Actual_Result, Predicted_Result))

0.6578947368421053


In [95]:
# Creating Model with Hypothesis Two

sess = tf_reset()

def create_model():
    # create inputs
    input_ph = tf.placeholder(dtype=tf.float32, shape=[None, 8]) # Since we know the shape of the data we can manually input
    output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 1])

    # create variables
    W0 = tf.get_variable(name='W0', shape=[8, 20], initializer=tf.contrib.layers.xavier_initializer())
    W1 = tf.get_variable(name='W1', shape=[20, 20], initializer=tf.contrib.layers.xavier_initializer())
    W2 = tf.get_variable(name='W2', shape=[20, 1], initializer=tf.contrib.layers.xavier_initializer())

    b0 = tf.get_variable(name='b0', shape=[20], initializer=tf.constant_initializer(0.))
    b1 = tf.get_variable(name='b1', shape=[20], initializer=tf.constant_initializer(0.))
    b2 = tf.get_variable(name='b2', shape=[1], initializer=tf.constant_initializer(0.))

    weights = [W0, W1, W2]
    biases = [b0, b1, b2]
    activations = [tf.nn.relu, tf.nn.relu, None]

    # create computation graph
    layer = input_ph
    for W, b, activation in zip(weights, biases, activations):
        layer = tf.matmul(layer, W) + b
        if activation is not None:
            layer = activation(layer)
    output_pred = layer
    
    return input_ph, output_ph, output_pred
    
input_ph, output_ph, output_pred = create_model()
    
# create loss
mse = tf.reduce_mean(0.5 * tf.square(output_pred - output_ph))

# create optimizer
# opt = tf.train.AdamOptimizer().minimize(mse)
opt = tf.train.GradientDescentOptimizer(0.01).minimize(mse)

# initialize variables
sess.run(tf.global_variables_initializer())
# create saver to save model variables
saver = tf.train.Saver()

# run training
batch_size = 40
for training_step in range(40000):
    # get a random subset of the training data
    indices = np.random.randint(low=0, high=len(data_train), size=batch_size)
    input_batch = data_train[indices]
    output_batch = label_train[indices]
    
    # run the optimizer and get the mse
    _, mse_run = sess.run([opt, mse], feed_dict={input_ph: input_batch, output_ph: output_batch})
    
    # print the mse every so often
    if training_step % 5000 == 0:
        print('{0:04d} mse: {1:.3f}'.format(training_step, mse_run))
        saver.save(sess, '/tmp/model_diabetes2.ckpt')

0000 mse: 0.470
5000 mse: 0.056
10000 mse: 0.058
15000 mse: 0.052
20000 mse: 0.055
25000 mse: 0.040
30000 mse: 0.048
35000 mse: 0.063


In the above model,   Optimizer = GradientDescentOptimizer, batch size=40 and iterations=40000

In [93]:
# Prediction of  Result with the Hypothesis 2

sess = tf_reset()

# create the model
input_ph, output_ph, output_pred = create_model()

# restore the saved model
saver = tf.train.Saver()
saver.restore(sess, "/tmp/model_diabetes2.ckpt")
predictions= sess.run(output_pred, feed_dict={input_ph: data_test})
pTClass = np.round(predictions)
Predicted_Result = np.ravel(abs(pTClass)).astype(int)
Actual_Result= np.ravel(label_test)
print('Predicted test classes :')
# print(np.ravel(abs(pTClass)))
print(Predicted_Result)
print('Actual test classes:')
print(Actual_Result)


INFO:tensorflow:Restoring parameters from /tmp/model_diabetes2.ckpt
Predicted test classes :
[0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 0 1 0 0 1 1 1 1 1 2 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0
 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0
 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1
 1 1 0 0]
Actual test classes:
[0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0
 1 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1
 1 1 1 0]


In [94]:
from sklearn import metrics
print(metrics.accuracy_score(Actual_Result, Predicted_Result))

0.7236842105263158


The tuning of parameters like  optimizer , batch size and number of iterations has helped in increasing the accuracy from 65.78% to 72.36 %. 


